<div style="text-align: justify; padding:5px; background-color:rgb(252, 253, 255); border: 1px solid lightgrey; padding-left: 1em; padding-right: 1em;">
    <font color='red'>To begin: Click anywhere in this cell and press <kbd>Run</kbd> on the menu bar. This executes the current cell and then highlights the next cell. There are two types of cells. A <i>text cell</i> and a <i>code cell</i>. When you <kbd>Run</kbd> a text cell (<i>we are in a text cell now</i>), you advance to the next cell without executing any code. When you <kbd>Run</kbd> a code cell (<i>identified by <span style="font-family: courier; color:black; background-color:white;">In[ ]:</span> to the left of the cell</i>) you advance to the next cell after executing all the Python code within that cell. Any visual results produced by the code (text/figures) are reported directly below that cell. Press <kbd>Run</kbd> again. Repeat this process until the end of the notebook. <b>NOTE:</b> All the cells in this notebook can be automatically executed sequentially by clicking <kbd>Kernel</kbd><font color='black'>→</font><kbd>Restart and Run All</kbd>. Should anything crash then restart the Jupyter Kernal by clicking <kbd>Kernel</kbd><font color='black'>→</font><kbd>Restart</kbd>, and start again from the top.
        
</div>

<div style="text-align: justify; padding:5px; background-color:rgb(252, 253, 255); border: 1px solid lightgrey; padding-left: 1em; padding-right: 1em;">
<img src="https://github.com/CIMCB/MetabComparisonBinaryML/blob/master/cimcb_logo.png?raw=true" width="180px" align="right" style="padding: 20px">

<a id="introduction"></a>

<h1> Metabolomics Data Visualisation Workflow for PLS-DA</h1>

<br>
<br>
<br>
<p  style="text-align: justify">This Jupyter Notebook described a metabolomics data analysis and visualisation workflow for partial least squares regression (a.k.a. projection to latent structure) with a binary classification outcome.</p>

<p style="text-align: justify">This computational workflow is described using a previously published NMR dataset by <a href="https://www.nature.com/articles/bjc2015414">Chan et al. (2016)</a>.The study compared the urine metabolomic profile comparison across patients characterised as Gastric Cancer (GC; n=43), Benign Gastric Disease (BN; n=40), and Healthy Control (HE; n=40) using 149 named metabolites. For the purpose of this computational workflow, we compare only the GC vs HE samples in a binary discriminant analysis. The deconvolved and annotated data from this study is deposited on <a href="https://www.metabolomicsworkbench.org/">Metabolomics Workbench</a> (Study ID: ST001047), and can be accessed directly via its Project DOI: <a href="http://dx.doi.org/DOI:10.21228/M8B10B">10.21228/M8B10B</a>. The Excel file used in this workflow can be accessed via the following link: <a href="https://github.com/CIMCB/MetabComparisonBinaryML/blob/master/dynamic/data/ST001047.xlsx?raw=true">ST001047.xlsx</a>.</p>

<p style="text-align: justify">This computational workflow requires a dataset to be in, or converted to, a previously described standardised Excel file format <a href="https://doi.org/10.1007/s11306-019-1588-0">(Mendez et al. 2019)</a>. This format uses the Tidy Data Framework <a href="https://www.jstatsoft.org/index.php/jss/article/view/v059i10/v59i10.pdf">(Wickham, 2014)</a>, where each row represents an observation (e.g. sample) and each column represents a variable (e.g. age or metabolite). Each excel file (per study) contains two sheets; a data sheet and a peak sheet. The data sheet contains the metabolite concentration together with the metadata associated for each observation (requiring the inclusion of the columns: Idx, SampleID, and Class). The peak sheet contains the additional metadata that pertains to the metabolites in the data sheet (requiring the inclusion of the columns: Idx, Name, and Label). The standardisation of this format allows for the efficient re-use of this computational workflow.</p>

The steps included in this data analysis and visualisation workflow are: 
<br>

1. [Import Packages](#1)<br>
2. [Load Data & Peak Sheet](#2)<br>
3. [Extract X & Y](#3)<br>
4. [Hyperparameter Optimisation](#4)<br>
    4.1. [Plot R² & Q²](#4.1)<br>
    4.2. [Plot Projections: Full & CV](#4.2)<br>
    4.3. [Permutation Test](#4.3)<br>
5. [Build Model](#5)<br>
6. [Bootstrap Aggregation (Bagging) of Model](#6)<br>
7. [Model Evaluation](#7)<br>
8. [Model Visualisation](#8)<br>
    8.1. [Plot Projections](#8.1)<br>
    8.2. [Plot Loadings](#8.2)<br>
    8.3. [Plot Feature Importance](#8.3)<br>
9. [Export Results](#9)<br>

</div>

<div style="background-color:rgb(255, 250, 250); padding:5px; padding-left: 1em; padding-right: 1em;">
    
<a id="1"></a>
<h2 style="text-align: justify">1. Import Packages</h2>

<p style="text-align: justify"><em>Packages</em> provide additional tools that extend beyond the basic functionality of the Python programming. Prior to usage, <em>packages</em> need to be imported into the Jupyter environment. The following <em>packages</em> need to be imported for this computional workflow:<br></p>

<ul>
<li style="text-align: justify"><a href="http://www.numpy.org/"><code>numpy</code></a>: a standard package primarily used for the manipulation of arrays</li>

<li style="text-align: justify"><a href="https://pandas.pydata.org/"><code>pandas</code></a>: a standard package primarily used for the manipulation of data tables</li>

<li style="text-align: justify"><a href="https://github.com/CIMCB/cimcb"><code>cimcb</code></a>: a library of helpful functions and tools provided by the authors</li>
</ul>

<br>

</div>

In [1]:
import numpy as np
import pandas as pd
import cimcb as cb

print('All packages successfully loaded')

Using TensorFlow backend.


All packages successfully loaded


<div style="background-color:rgb(255, 250, 250); padding:5px; padding-left: 1em; padding-right: 1em;">

<a id="2"></a>
<h2 style="text-align: justify">2. Load Data & Peak Sheet</h2>

<p style="text-align: justify">This CIMCB helper function <code>load_dataXL()</code> loads the <em>Data</em> and <em>Peak</em> sheet from an Excel file. In addition, this helper function checks that the data is in the standardised Excel file format described <a href=#introduction>above</a>. After the initial checks, <code>load_dataXL()</code> outputs two individual Pandas DataFrames (i.e. tables) called <code>DataTable</code> and <code>PeakTable</code> from the Excel file <a href="https://github.com/CIMCB/MetabComparisonBinaryML/blob/master/dynamic/data/ST001047.xlsx?raw=true">ST001047.xlsx</a>. This helper function requires values for the following parameters:</p>
<ul>
    <li><code>filename</code>: the name of the excel file (.xlsx file)</li>
    <li><code>DataSheet</code>: the name of the data sheet in the file</li>
    <li><code>PeakSheet</code>: the name of the peak sheet in the file</li>
</ul>   
<br>

</div>

In [19]:
home = 'data/'
file = 'ST001047.xlsx'

DataTable,PeakTable = cb.utils.load_dataXL(filename=home + file, DataSheet='Data', PeakSheet='Peak')

Loadings PeakFile: Peak
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 140 TOTAL PEAKS: 149
Done!


<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<a id="3"></a>
<h2 style="text-align: justify">3. Extract X & Y</h2>

<p style="text-align: justify">Prior to performing any statistical or machine learning modelling, it is best practice to assess the quality of the data and remove metabolites that lack reproducible measurements  <a href="https://link.springer.com/article/10.1007/s11306-018-1367-3">(Broadhurst et al. 2018)</a>. In this dataset <a href="https://github.com/CIMCB/MetabComparisonBinaryML/blob/master/dynamic/data/ST001047.xlsx?raw=true">ST001047.xlsx</a>, we can find that the QC-RSD and percentage of missing value has been previously calculated (refer to the peak sheet). In this Jupyter Notebook, we remove all metabolites that do not meet the following criteria:</p>

<ul>
<li style="text-align: justify">QC-RSD less than 20% </li>

<li style="text-align: justify">Fewer than 10% of values are missing</li>
</ul>

<br>
<p style="text-align: justify">The following steps are needed to: <b>(a)</b> extract the binary outcome (i.e. GC vs. HE) and <b>(b)</b> extract, transform, and scale the metabolite data matrix, with missing values imputed.</p>

<ul>
    
<li style="text-align: justify">Create a subset of <code>DataTable</code> called <code>DataTable2</code>, only with samples in the Class “GC” or “HE”</li>
    
<li style="text-align: justify">Set <code>Y</code> to a list (or 1D array) of binary outcomes based on the Class column from <code>DataTable2</code> (“GC”=1 and “HE”=0)</li>

<li style="text-align: justify">Create the variable <code>peaklist</code> to hold the names (M1...Mn) of the metabolites to be used</li>

<li style="text-align: justify">Using this <code>peaklist</code>, extract all corresponding columns (i.e. metabolite data) from <code>DataTable2</code>, and place it in matrix <code>X</code></li>

<li style="text-align: justify">Log-transform the values in <code>X</code></li>

<li style="text-align: justify">Using the helper function <code>cb.utils.scale()</code>, scale the log-transformed data to the unit variance (a.k.a. auto scaling).</li>

<li style="text-align: justify">Impute the missing values by using a <em>k</em>-nearest neighbour approach (with three neighbours) using the helper function <code>cb.utils.knnimpute()</code> to give the final matrix, <code>XTknn</code></li>
    
</ul>

<br>
</div>

In [3]:
# Clean PeakTable
RSD = PeakTable['QC_RSD']
PercMiss = PeakTable['Perc_missing']
PeakTableClean = PeakTable[(RSD < 20) & (PercMiss < 10)]

# Select Subset of Data
DataTable2 = DataTable[(DataTable.Class == "GC") | (DataTable.Class == "HE")]

# Create a Binary Y Vector 
Outcomes = DataTable2['Class']
Y = [1 if outcome == 'GC' else 0 for outcome in Outcomes]
Y = np.array(Y)

# Extract and Scale Metabolite Data 
peaklist = PeakTableClean['Name']
XT = DataTable2[peaklist]
XTlog = np.log(XT)
XTscale = cb.utils.scale(XTlog, method='auto')
XTknn = cb.utils.knnimpute(XTscale, k=3)

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<a id="4"></a>
<h2 style="text-align: justify"> 4. Hyperparameter Optimisation </h2>

<p style="text-align: justify">The CIMCB helper function <code>cb.cross_val.kfold()</code> is used to carry out <em>k</em>-fold cross-validation (<em>k</em>=5) on a set of PLS-DA models with varying number of latent variables (1 to 6) to determine the optimal number. In <em>k</em>-fold cross-validation, the original dataset is randomly split into k sized folds and subsequently trained for <em>k</em> iterations, where the model is trained on 1 – <em>k</em> folds and tested on the <em>k</em> fold <a href='http://ai.stanford.edu/~ronnyk/accEst.pdf'>(Kohavi 1995)</a>. This helper function requires values for the following parameters:</p>
    
<ul>
    <li><code>model</code>: the class of model used by the function, <code>cb.model.PLS_SIMPLS</code></li>
    <li><code>X</code>: the metabolite data matrix, <code>XTknn</code></li>
    <li><code>Y</code>: the binary outcome vector, <code>Y</code></li>
    <li><code>param_dict</code>: a dictionary, <code>param_dict</code>, that describes all key:value pairs to search, with the key name corresponding to the hyperparameter in the model class and the value as the list of possible values</li>
    <li><code>folds</code>: the number of folds in the <em>k</em>-fold cross validation</li>
    <li><code>n_mc</code>: the number of Monte Carlo repetitions of the <em>k</em>-fold CV</li>
</ul>
<br>
</div>

In [4]:
# Parameter Dictionary
param_dict = {'n_components': [1, 2, 3, 4, 5, 6]}                   

# Initialise
cv = cb.cross_val.kfold(model=cb.model.PLS_SIMPLS,                      
                                X=XTknn,                                 
                                Y=Y,                               
                                param_dict=param_dict,                   
                                folds=5,
                                n_mc=100)                                

# Run 
cv.run()  

Number of cores set to: 8
Running ...


100%|██████████| 600/600 [00:08<00:00, 73.30it/s]


Time taken: 0.14 minutes with 8 cores
Done!


<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<a id="4.1"></a>
<h3 style="text-align: justify"> 4.1. Plot R² & Q²</h3>

<p style="text-align: justify">When <code>cv.plot(metric='r2q2')</code> is run, 2 plots of $R^2$ and $Q^2$ statistics are displayed: (a) the absolute difference of ($R^2 - Q^2$) vs. $Q^2$, and (b) $R^2$ and $Q^2$ against the number of latent variables. The optimal number of hyperparameters is selected based on the point of inflection in figure b, or if a clear inflection point is not present, where | $R^2 - Q^2$ | = 0.2.  Note, the $R^2$ is the mean coefficient of determination for the full dataset, and the $Q^2$ is the mean coefficient of determination for cross-validated prediction dataset over the 100 Monte Carlo repetitions. The following parameters of <code>cv.plot(metric='r2q2')</code> can be altered:</p>
<ul>
    <li><code>metric</code>: the metric used for the plots (default = 'r2q2'). Alternative metrics include 'auc', 'acc', 'f1score', 'prec', 'sens', and 'spec'.
    <li><code>ci</code>: the confidence interval in figure b (default = 95)
</ul>

<br>
</div>

In [18]:
cv.plot(metric='r2q2', ci=95)

Loading BokehJS ...

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<a id="4.2"></a>
<h3 style="text-align: justify"> 4.2. Plot Projections: Full & CV </h3>

<p style="text-align: justify">When <code>cv.run()</code> followed by <code>cv.plot_projections()</code> are run the results are displayed as a combination of scatterplots, distributions plots, and ROC curves for each combination of latent variables. The following paramaters of the method can be adjusted to better interrogate the optimal number of latent variables:
 <ul>
    <li><code>components</code>: LVs to plot (default = None ; plot all components)</li>
     <li><code>scatter_show</code>: Series of dots to plot in scatterplot (default = None). Alternative values include "Inner", "Full", "CV" and "All"</li>
 </ul>
</p>
</div>

In [23]:
cv.plot_projections(components=[1,3,2],
                    scatter_show="None")

Loading BokehJS ...

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<a id="4.3"></a>
<h3 style="text-align: justify"> 4.3. Permutation Test</h3>

<p style="text-align: justify">After a model has been trained, the <code>.permutation_test()</code> method can be used to assess the reliability of the trained model (after selecting the number of latent variables). For the permutation test, the metabolite data matrix is randomised (permuted or 'shuffled'), while the Y (i.e. outcome) is fixed, and subsequently trained and tested on this randomised data <a href='https://link.springer.com/article/10.1007/s11306-011-0330-3'>(Szymańska et al. 2012)</a>. This process is repeated (in this case, n=100) to construct a distribution to fairly access the model. For a dataset with features that have with no meaningful contribution, we would expect a similar $R^2$ and $Q^2$ to a randomised dataset, while for a dataset with features with meaningful contribution, we would expect a $R^2$ and $Q^2$ significantly higher than that of the randomised dataset. When <code>.permutation_test()</code> is run, 2 plots are displayed: (a) $R^2$ and $Q^2$ against "correlation of permuted data against original data", and (b) probability density functions for $R^2$ and $Q^2$, with the $R^2$ and $Q^2$ values found for the model trained on original data presented as ball-and-stick. The following parameter value of <code>.permutation_test()</code> can be altered: 

<ul>
    <li><code>nperm</code>: the number of permutations. (default = 100)
</ul>

<br>
</div>

In [20]:
model = cb.model.PLS_SIMPLS(n_components=2)
model.train(XTknn, Y)
model.permutation_test(nperm=100)

Permutation Resample: 100%|██████████| 100/100 [00:01<00:00, 87.88it/s]


Loading BokehJS ...

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<h2 style="text-align: justify"> 5. Build Model </h2>

<p style="text-align: justify">Redoing Text<p>

</div>

In [8]:
# Build Model
model = cb.model.PLS_SIMPLS(n_components=2) 

# Train Model
Ypred = model.train(XTknn, Y)
#Ypred_test = model.test(XTknn)

# Evaluate Model 
#model.evaluate(cutoffscore=0.5) 

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<h3 style="text-align: justify"> 5.1. Bootstrap Aggregation (Bagging) </h3>

<p style="text-align: justify">Redoing Text<p>

</div>

In [9]:
bootmodel = cb.bootstrap.BC(model, bootnum=100)
bootmodel.run()

Bootstrap Resample: 100%|██████████| 100/100 [00:00<00:00, 893.57it/s]


<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<h3 style="text-align: justify"> 5.2. Bootstrap Evaluation </h3>

<p style="text-align: justify">Redoing Text<p>


</div>

In [10]:
#### Testing a few options: ####
# parametric = True / False / None / 1
# True: Upper limit exactly mirrors lower limit
# False: Same as True, except if Upper CI was already 1, upper limit remains 1 at given point
# None: Upper limit is always 1
# 1: No upper limit

# BC = True / False; whether to do bias-correction for ROC Curve

bootmodel.evaluate(parametric=None, bc=True) # Options: parametric = True / False 

Loading BokehJS ...

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<h3 style="text-align: justify"> 5.3. Visualisation </h3>

<p style="text-align: justify">Redoing Text<p>

</div>

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<h3 style="text-align: justify"> 5.3.1 Plot Projections </h3>

<p style="text-align: justify">Redoing Text<p>

</div>

In [11]:
# Scatter_show option: "None", "Inner", "IB", "OOB", "All"

bootmodel.plot_projections(scatter_show='None',
                           legend=True,
                           label=DataTable2.Class)

Loading BokehJS ...

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<h3 style="text-align: justify"> 5.3.2 Plot Loadings </h3>

<p style="text-align: justify">Redoing Text<p>

</div>

In [12]:
bootmodel.plot_loadings(PeakTable,
                        peaklist,
                        ylabel='Label',  # change ylabel to 'Name' 
                        sort=True)      # change sort to False

Loading BokehJS ...

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<h3 style="text-align: justify"> 5.3.3. Plot Feature Importance (Coefficient plot and VIP) for PLS-DA model </h3>

<p style="text-align: justify">Redoing Text<p>

</div>

In [13]:
peakSheet_featureimportance = bootmodel.plot_featureimportance(PeakTable,
                                         peaklist,
                                         ylabel='Label',  # change ylabel to 'Name' 
                                         sort=False,
                                         sort_ci=True)      # change sort to False

Loading BokehJS ...

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<h3 style="text-align: justify"> 7. Export results table to Excel </h3>

<p style="text-align: justify">Redoing Text<p>

</div>

In [14]:
# Create an empty excel workbook
writer = pd.ExcelWriter("PLSDA_ST001047.xlsx")     # provide the filename for the Excel file

# Add each dataframe to the workbook in turn, as a separate worksheet
peakSheet_featureimportance.to_excel(writer, sheet_name='Peaksheet', index=False)

# Write the Excel workbook to disk
writer.save()

print("Done!")

Done!
